In [1]:
#SETTING PATH 
path="/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"
previousResults="/sharedFolder/Results/Hotspot/Revision1/FULL/"
results="/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"
dir.create(results,recursive=TRUE)
riderPath="/home/Riderv2/"

In [2]:
#LOAD USEFULL FUNCTIONS 
noExt=function(x){
return(tools::file_path_sans_ext(x))
}
bnoExt=function(x){
return(basename(tools::file_path_sans_ext(x)))
}
fNumber=function(x){
sapply(x,FUN=function(x){
strsplit(x,"__")[[1]][2]
})
}
rRider=function(x){
sapply(x,FUN=function(x){
strsplit(x,"-RIDER")[[1]][1]
})
}
# centering with 'scale()'
center_scale <- function(x) {
    scale(x, scale = FALSE)
}

In [3]:
#LOAD LIBRARIES 
library("argparser");
library(gtools);
library("GenomicRanges");
library("ggVennDiagram");
options(warn=-1)
library(GenomicRanges)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    I, expand.grid, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



In [32]:
library(gtools)
library(GenomicRanges)

# Funzioni utili
noExt <- function(x) tools::file_path_sans_ext(x)
fNumber <- function(x) strsplit(x, "__")[[1]][2]

# Path fissi
path <- "/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"
base_results <- "/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"
dir.create(base_results, recursive = TRUE, showWarnings = FALSE)

thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {

  message("\n--- Processing threshold: ", jamesonFirst, " ---")
  
  # Percorsi
  previousResults_thr <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst)
  results_thr <- paste0(base_results, jamesonFirst, "/")
  dir.create(results_thr, recursive = TRUE, showWarnings = FALSE)

  # Carica e filtra il database
  db_path <- paste0(previousResults_thr, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  Database <- Database[grep("Idelalisib", Database$HotspotBelonging), ]
  
  # Filtra i .bed di Idelalisib
  a <- list.files(path, pattern = ".bed", recursive = TRUE)
  a <- mixedsort(a)
  a2 <- sapply(a, function(x) strsplit(x, ".bed")[[1]][1])
  a2 <- sapply(a2, function(x) strsplit(x, "__")[[1]][2])
  idx <- grep("Idelalisib", a2)
  a <- a[idx]
  a2 <- a2[idx]

  # Inizializzazione matrici
  hotspots <- unique(Database$hotspotID)
  hgtgts_count <- matrix(0, ncol = length(hotspots), nrow = length(a2))
  colnames(hgtgts_count) <- hotspots
  rownames(hgtgts_count) <- a2
  hgtgts_rpm <- hgtgts_count

  # Conta overlaps
  for (j in a) {
    message(" > File: ", j)
    temp <- read.table(paste0(path, "/", j), sep = "\t")
    tempGG <- GRanges(seqnames = temp[,1], ranges = IRanges(start = as.numeric(temp[,2]), end = as.numeric(temp[,3])))
    name <- noExt(as.vector(fNumber(j)))
    
    for (i in seq_len(nrow(Database))) {
      gg <- GRanges(seqnames = Database[i,1], ranges = IRanges(start = as.numeric(Database[i,2]), end = as.numeric(Database[i,3])))
      over <- countOverlaps(gg, tempGG)
      hgtgts_count[name, toString(Database[i,7])] <- over
      hgtgts_rpm[name, toString(Database[i,7])] <- (over / nrow(temp)) * 1e6
    }
  }

  # Scrivi file "plain"
  write.table(t(hgtgts_count), paste0(results_thr, "count_hgtgtsHeatmap.csv"), col.names = NA, sep = ",")
  write.table(t(hgtgts_rpm), paste0(results_thr, "rpm_hgtgtsHeatmap.csv"), col.names = NA, sep = ",")

  # Annotazioni
  annotations <- c()
  for (n in colnames(hgtgts_count)) {
    ann <- unique(Database[which(Database$hotspotID == n), ]$annotation)
    if (is.null(ann) || ann == "NULL") {
      ann <- paste(unique(Database[which(Database$hotspotID == n), ]$Chromosome), ":",
                   unique(Database[which(Database$hotspotID == n), ]$ChrStart), "-",
                   unique(Database[which(Database$hotspotID == n), ]$ChrEnd), sep = "")
    }
    annotations <- c(annotations, ann)
  }

  colnames(hgtgts_count) <- annotations
  colnames(hgtgts_rpm) <- annotations

  # Scrivi file ANNOTATED
  write.table(t(hgtgts_count), paste0(results_thr, "count_hgtgtsHeatmapANNOTATED.csv"), col.names = NA, sep = ",")
  write.table(t(hgtgts_rpm), paste0(results_thr, "rpm_hgtgtsHeatmapANNOTATED.csv"), col.names = NA, sep = ",")
}



--- Processing threshold: 5 ---

 > File: 7__MEC-1_Idelalisib_DMSO.bed

 > File: 8__MEC-1_Idelalisib_Tazemetostat.bed

 > File: 9__MEC-1_Idelalisib_Valemetostat.bed


--- Processing threshold: 6 ---

 > File: 7__MEC-1_Idelalisib_DMSO.bed

 > File: 8__MEC-1_Idelalisib_Tazemetostat.bed

 > File: 9__MEC-1_Idelalisib_Valemetostat.bed


--- Processing threshold: 7 ---

 > File: 7__MEC-1_Idelalisib_DMSO.bed

 > File: 8__MEC-1_Idelalisib_Tazemetostat.bed

 > File: 9__MEC-1_Idelalisib_Valemetostat.bed



In [33]:
library(eulerr)
library(grid)
library(gtools)

types <- c("Idelalisib")
conditions <- c("DMSO", "Tazemetostat", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  
  for (j in types) {
    subBase=Database[grep(j,Database[,6]),]
    x <- list()
    for (i in conditions) {
    x[[i]]=subBase[grep(i,subBase[,6]),7]
    }
    
    venn_counts <- c(
      "DMSO" = length(setdiff(setdiff(x$DMSO, x$Tazemetostat), x$Valemetostat)),
      "Tazemetostat" = length(setdiff(setdiff(x$Tazemetostat, x$DMSO), x$Valemetostat)),
      "Valemetostat" = length(setdiff(setdiff(x$Valemetostat, x$Tazemetostat), x$DMSO)),
      "DMSO&Tazemetostat&Valemetostat" = length(Reduce(intersect, x)),
      "DMSO&Tazemetostat" = length(setdiff(intersect(x$DMSO, x$Tazemetostat), x$Valemetostat)),
      "DMSO&Valemetostat" = length(setdiff(intersect(x$DMSO, x$Valemetostat), x$Tazemetostat)),
      "Tazemetostat&Valemetostat" = length(setdiff(intersect(x$Tazemetostat, x$Valemetostat), x$DMSO))
    )
    
    fit <- euler(venn_counts)
    
    # Output path
    out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
    dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)
    
    # PDF device con grid-compatible output
    cairo_pdf(filename = paste0(out_dir, "venn_diagram.pdf"), width = 6, height = 6)
    grid.draw(plot(fit, quantities = TRUE, main = paste("Idelalisib Hotspot - Threshold", jamesonFirst)))
    dev.off()
  }
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [34]:
library(eulerr)
library(grid)

conditions <- c("DMSO", "Tazemetostat", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  # Load and filter database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  
  subBase <- Database[grep("Idelalisib", Database[,6]), ]  # Usa correttamente la colonna 6

  # Costruzione delle liste per le condizioni
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase[,6]), "hotspotID"]
  }

  # Sezioni di interesse
  DMSOSpec   <- setdiff(setdiff(x$DMSO, x$Valemetostat), x$Tazemetostat)
  TazeSpec   <- setdiff(setdiff(x$Tazemetostat, x$DMSO), x$Valemetostat)
  ValeSpec   <- setdiff(setdiff(x$Valemetostat, x$DMSO), x$Tazemetostat)
  TazVale    <- setdiff(intersect(x$Tazemetostat, x$Valemetostat), x$DMSO)
  TazDMSO    <- setdiff(intersect(x$Tazemetostat, x$DMSO), x$Valemetostat)
  DMSOVale   <- setdiff(intersect(x$Valemetostat, x$DMSO), x$Tazemetostat)
  common     <- Reduce(intersect, x)

  # Venn diagram tra DMSO e Tazemetostat
  fit1 <- euler(c(
    "DMSO" = length(setdiff(x$DMSO, x$Tazemetostat)),
    "Tazemetostat" = length(setdiff(x$Tazemetostat, x$DMSO)),
    "DMSO&Tazemetostat" = length(intersect(x$DMSO, x$Tazemetostat))
  ))

  # Path risultati
  out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

  # Salva Venn diagram
  cairo_pdf(paste0(out_dir, "venn_dmsovstaze.pdf"), width = 6, height = 6)
  grid.draw(plot(fit1, quantities = TRUE, main = paste("Idelalisib Hotspot - DMSO vs Tazemetostat - Threshold", jamesonFirst)))
  dev.off()

  # Scrivi i file delle liste
  res2 <- paste0(out_dir, "HotspotIDList/")
  dir.create(res2, showWarnings = FALSE)

  write.table(unique(union(x$Tazemetostat, x$DMSO)), paste0(res2, "TazeFULL.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
  write.table(unique(union(x$Valemetostat, x$DMSO)), paste0(res2, "ValeFULL.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
  write.table(intersect(x$Tazemetostat, x$DMSO), paste0(res2, "Tazecommon.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
  write.table(intersect(x$Valemetostat, x$DMSO), paste0(res2, "Valecommon.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [35]:
library(eulerr)
library(grid)

conditions <- c("Tazemetostat", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  # Load and filter database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  subBase <- Database[grep("Idelalisib", Database[,6]), ]
  
  # Costruzione liste
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase[,6]), "hotspotID"]
  }

  # Per comodità di sintassi
  j2 <- x
  
  # Costruzione eulerr
  fit2 <- euler(c(
    "Tazemetostat" = length(setdiff(j2$Tazemetostat, j2$Valemetostat)),
    "Valemetostat" = length(setdiff(j2$Valemetostat, j2$Tazemetostat)),
    "Tazemetostat&Valemetostat" = length(intersect(j2$Tazemetostat, j2$Valemetostat))
  ))

  # Salvataggio
  out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

  cairo_pdf(paste0(out_dir, "venn_tazevsvale.pdf"), width = 6, height = 6)
  grid.draw(plot(fit2, quantities = TRUE, main = paste("Idelalisib Hotspot - Taze vs Vale - Threshold", jamesonFirst)))
  dev.off()
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [36]:
library(eulerr)
library(grid)

conditions <- c("DMSO", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  # Carica e filtra il database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  subBase <- Database[grep("Idelalisib", Database[,6]), ]

  # Crea le liste hotspot
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase[,6]), "hotspotID"]
  }

  j2 <- x  # alias

  # Costruisci Venn diagram
  fit3 <- euler(c(
    "DMSO" = length(setdiff(j2$DMSO, j2$Valemetostat)),
    "Valemetostat" = length(setdiff(j2$Valemetostat, j2$DMSO)),
    "DMSO&Valemetostat" = length(intersect(j2$DMSO, j2$Valemetostat))
  ))

  # Salva il plot
  out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

  cairo_pdf(paste0(out_dir, "venn_dmsovsvale.pdf"), width = 6, height = 6)
  grid.draw(plot(fit3, quantities = TRUE, main = paste("Idelalisib Hotspot - DMSO vs Vale - Threshold", jamesonFirst)))
  dev.off()
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [48]:
library(pheatmap)
library(vioplot)
library(ggVennDiagram)
library(ggplot2)
library(GenomicRanges)
library(eulerr)
library(grid)

rider2 <- "/sharedFolder/rider/RIDER_v0.2.jar"
path <- "/sharedFolder/Data/1_HTGTS/13_Revision1/"
base_results <- "/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- RIDER Filtering & Venn Analysis - Threshold", jamesonFirst, "---\n")
  
  results <- file.path(base_results, as.character(jamesonFirst))
  previousResults <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/")
  db_path <- paste0(previousResults, "Database_annotated.csv")
  plots <- file.path(results, "rpmPlot")
  res2 <- file.path(results, "HotspotIDList")
  
  dir.create(plots, recursive = TRUE, showWarnings = FALSE)
  dir.create(file.path(plots, "filtered"), showWarnings = FALSE)
  dir.create(res2, showWarnings = FALSE)

  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[Database$Category == "AID_off_target", ]
  gg <- GRanges(seqnames = Database$Chromosome, ranges = IRanges(start = Database$ChrStart, end = Database$ChrEnd))

  # --- Generate RPM plots before filtering ---
  subBase <- Database[grep("Idelalisib", Database$HotspotBelonging), ]
  conditions <- c("DMSO", "Tazemetostat", "Valemetostat")
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase$HotspotBelonging), "hotspotID"]
  }
  j2 <- x

  a <- as.matrix(read.table(file.path(results, "rpm_hgtgtsHeatmap.csv"), header = TRUE, row.names = 1, sep = ","))

  # DMSO vs Tazemetostat
  temp <- a[sapply(setdiff(j2$DMSO, j2$Tazemetostat), toString), c(1,2), drop = FALSE]
  temp2 <- a[sapply(intersect(j2$DMSO, j2$Tazemetostat), toString), c(1,2), drop = FALSE]
  temp3 <- a[sapply(setdiff(j2$Tazemetostat, j2$DMSO), toString), c(1,2), drop = FALSE]
  write.table(rbind(temp, temp2, temp3), file.path(plots, "Ide_DMSOTazemetostat.csv"), col.names = NA, sep = ",", quote = TRUE)

  # DMSO vs Valemetostat
  temp <- a[sapply(setdiff(j2$DMSO, j2$Valemetostat), toString), c(1,3), drop = FALSE]
  temp2 <- a[sapply(intersect(j2$DMSO, j2$Valemetostat), toString), c(1,3), drop = FALSE]
  temp3 <- a[sapply(setdiff(j2$Valemetostat, j2$DMSO), toString), c(1,3), drop = FALSE]
  write.table(rbind(temp, temp2, temp3), file.path(plots, "Ide_DMSOValemetostat.csv"), col.names = NA, sep = ",", quote = TRUE)

  # --- RIDER Filtering ---
  filePlots <- list.files(plots, pattern = "^Ide_.*\\.csv$", full.names = TRUE)
  fileNames <- basename(filePlots)
  toRemoveList <- setNames(vector("list", length(filePlots)), fileNames)

  for (i in 1:(length(conditions) - 1)) {
    for (j in (i + 1):length(conditions)) {
      cond1 <- conditions[i]
      cond2 <- conditions[j]
      if ((cond1 == "Tazemetostat" && cond2 == "Valemetostat") || (cond1 == "Valemetostat" && cond2 == "Tazemetostat")) next

      pair_name <- paste0("Ide_", cond1, cond2, ".csv")
      rpm_file <- file.path(plots, pair_name)
      if (!file.exists(rpm_file)) next
      output_dir <- file.path(plots, paste0(cond1, "_", cond2))
      dir.create(output_dir, showWarnings = FALSE)
      print(paste("java -jar", rider2,
                   "-task ValSite -s_file", path, "-e_file", path,
                   "-c_file", path,
                   "-genome hg38 -out_dir", output_dir))
      system(paste("java -jar", rider2,
                   "-task ValSite -s_file", path, "-e_file", path,
                   "-c_file", path,
                   "-genome hg38 -out_dir", output_dir))

      result_file <- list.files(output_dir, pattern = ".txt", full.names = TRUE)
      if (length(result_file) == 0) next

      compare <- read.table(result_file[1], header = TRUE, sep = "\t")
      toRemove <- c()
      for (n in which(compare[, 16] > 0.05)) {
        tempGG <- GRanges(seqnames = compare[n, 2], ranges = IRanges(start = compare[n, 3], end = compare[n, 4]))
        hits <- findOverlaps(tempGG, gg)
        toRemove <- c(toRemove, unique(Database[subjectHits(hits), "hotspotID"]))
      }

      toRemoveList[[pair_name]] <- intersect(sapply(toRemove, toString), rownames(read.table(rpm_file, header = TRUE, row.names = 1, sep = ",")))
    }
  }

  for (i in seq_along(filePlots)) {
    rpmPlot <- read.table(filePlots[i], header = TRUE, sep = ",", row.names = 1)
    rpmPlot <- rpmPlot[!(rownames(rpmPlot) %in% toRemoveList[[basename(filePlots[i])]]), ]
    write.table(rpmPlot, file.path(plots, "filtered", paste0("filtered_", basename(filePlots[i]))), col.names = NA, sep = ",")
  }

  # --- Generate final Venns and summary ---
  rpm <- read.table(file.path(results, "rpm_hgtgtsHeatmap.csv"), header = TRUE, row.names = 1, sep = ",")
  xTaze <- list(
    DMSO = union(setdiff(j2$DMSO, j2$Tazemetostat), intersect(j2$DMSO, j2$Tazemetostat)),
    Tazemetostat = union(setdiff(j2$Tazemetostat, j2$DMSO), intersect(j2$DMSO, j2$Tazemetostat))
  )
  xVale <- list(
    DMSO = union(setdiff(j2$DMSO, j2$Valemetostat), intersect(j2$DMSO, j2$Valemetostat)),
    Valemetostat = union(setdiff(j2$Valemetostat, j2$DMSO), intersect(j2$DMSO, j2$Valemetostat))
  )
  j2Filtered <- list(
    DMSO = union(xTaze$DMSO, xVale$DMSO),
    Tazemetostat = xTaze$Tazemetostat,
    Valemetostat = xVale$Valemetostat
  )

  venn_all <- euler(c(
    "DMSO" = length(setdiff(setdiff(j2Filtered$DMSO, j2Filtered$Tazemetostat), j2Filtered$Valemetostat)),
    "Tazemetostat" = length(setdiff(setdiff(j2Filtered$Tazemetostat, j2Filtered$DMSO), j2Filtered$Valemetostat)),
    "Valemetostat" = length(setdiff(setdiff(j2Filtered$Valemetostat, j2Filtered$Tazemetostat), j2Filtered$DMSO)),
    "DMSO&Tazemetostat&Valemetostat" = length(Reduce(intersect, j2Filtered)),
    "DMSO&Tazemetostat" = length(setdiff(intersect(j2Filtered$DMSO, j2Filtered$Tazemetostat), j2Filtered$Valemetostat)),
    "DMSO&Valemetostat" = length(setdiff(intersect(j2Filtered$DMSO, j2Filtered$Valemetostat), j2Filtered$Tazemetostat)),
    "Tazemetostat&Valemetostat" = length(setdiff(intersect(j2Filtered$Tazemetostat, j2Filtered$Valemetostat), j2Filtered$DMSO))
  ))
  cairo_pdf(file.path(plots, "venn_all_filtered.pdf"), width = 6, height = 6)
  plot(venn_all, quantities = TRUE, main = paste("Idelalisib Hotspot - All - Threshold", jamesonFirst))
  dev.off()

  # Barplot summary
  bar <- c(length(j2Filtered$DMSO), length(j2Filtered$Tazemetostat), length(j2Filtered$Valemetostat))
  names(bar) <- c("DMSO", "Tazemetostat", "Valemetostat")
  write.table(t(bar), file.path(plots, "filtered", "barplot.csv"), col.names = NA)
}



--- RIDER Filtering & Venn Analysis - Threshold 5 ---
[1] "java -jar /sharedFolder/rider/RIDER_v0.2.jar -task ValSite -s_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -e_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -c_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -genome hg38 -out_dir /sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib//5/rpmPlot/DMSO_Tazemetostat"
[1] "java -jar /sharedFolder/rider/RIDER_v0.2.jar -task ValSite -s_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -e_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -c_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -genome hg38 -out_dir /sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib//5/rpmPlot/DMSO_Valemetostat"

--- RIDER Filtering & Venn Analysis - Threshold 6 ---
[1] "java -jar /sharedFolder/rider/RIDER_v0.2.jar -task ValSite -s_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -e_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -c_file /sharedFolder/Data/1_HTGTS/13_Revision1/ -genome hg38 -out_dir /sharedFolder/Resul

In [59]:
thresholds <- c(5, 6, 7)
library("pheatmap")
library("vioplot")
library("ggVennDiagram")
library("ggplot2")
library("GenomicRanges")
library("eulerr")

rider2 <- "/sharedFolder/rider/RIDER_v0.2.jar"
path <- "/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"
base_results <- "/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"

for (jamesonFirst in thresholds) {
  message("\n--- Filtering with RIDER: Threshold ", jamesonFirst, " ---")
  
  results <- file.path(base_results, jamesonFirst)
  previousResults <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/")
  plots <- file.path(results, "rpmPlot")
  res2 <- file.path(results, "HotspotIDList")
  
  dir.create(plots, recursive = TRUE, showWarnings = FALSE)
  dir.create(file.path(plots, "filtered"), showWarnings = FALSE)
  dir.create(res2, showWarnings = FALSE)

  db_path <- paste0(previousResults, "Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[Database$Category == "AID_off_target", ]
  gg <- GRanges(seqnames = Database$Chromosome, ranges = IRanges(start = Database$ChrStart, end = Database$ChrEnd))

  filePlots <- list.files(plots, pattern = ".csv", full.names = TRUE)
  fileNames <- basename(filePlots)
  len <- length(filePlots)
  toRemoveList <- vector(mode = "list", length = len)
  names(toRemoveList) <- fileNames

  a <- list.files(path, recursive = TRUE, pattern = "Idelalisib")
  b <- list.dirs(previousResults, recursive = FALSE)
  b <- b[grep("Idelalisib", b)]

  conditions <- c("DMSO", "Taze", "Vale")

  for (i in conditions) {
    oneFile <- a[grep(i, a)]
    oneFileS <- b[grep(i, b)]
    pathone <- list.dirs(oneFileS, recursive = FALSE)
    oneFileS <- list.files(pathone, pattern = "bed", full.names = TRUE)

    for (j in conditions) {
      if (!((i == "Taze" && j == "Vale") || (j == "Taze" && i == "Vale")) && i != j) {
        twoFile <- a[grep(j, a)]
        twoFileS <- b[grep(j, b)]
        pathtwo <- list.dirs(twoFileS, recursive = FALSE)
        twoFileS <- list.files(pathtwo, pattern = "bed", full.names = TRUE)

        count <- intersect(grep(i, names(toRemoveList)), grep(j, names(toRemoveList)))
        toRemove <- c()

        plot_file <- filePlots[count]
        rpmPlot <- read.table(plot_file, header = TRUE, sep = ",", row.names = 1)

        output_dir <- file.path(plots, paste0(i, "_", j))
        dir.create(output_dir, showWarnings = FALSE)

        system(paste("java -jar", rider2, "-task ValSite -s_file", oneFileS, twoFileS,
                     "-e_file", file.path(path, oneFile),
                     "-c_file", file.path(path, twoFile),
                     "-genome hg38 -out_dir", output_dir))
        system(paste("chmod -R 777", output_dir))

        compare_file <- list.files(output_dir, pattern = ".txt", full.names = TRUE)
        compare <- read.table(compare_file, header = TRUE, sep = "\t")

        for (n in which(compare[, 16] > 0.05)) {
          tempGG <- GRanges(seqnames = compare[n, 2],
                            ranges = IRanges(start = compare[n, 3], end = compare[n, 4]))
          toRemove <- c(toRemove, unique(Database[queryHits(findOverlaps(tempGG, gg)), "hotspotID"]))
        }

        toRemove <- intersect(sapply(toRemove, toString), rownames(rpmPlot))

        if (length(toRemoveList[[count]]) == 0) {
          toRemoveList[[count]] <- toRemove
        } else {
          toRemoveList[[count]] <- intersect(toRemoveList[[count]], toRemove)
        }
      }
    }
  }

  for (i in seq_along(filePlots)) {
    rpmPlot <- read.table(filePlots[i], header = TRUE, sep = ",", row.names = 1)
    rpmPlot <- rpmPlot[!(rownames(rpmPlot) %in% toRemoveList[[i]]), ]
    write.table(rpmPlot, file.path(plots, "filtered", paste0("filtered_", basename(filePlots[i]))),
                col.names = NA, sep = ",")
  }

  # VENN FINAL ANALYSIS POST-RIDER
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[Database$Category == "AID_off_target", ]
  types <- c("Idelalisib")

  for (j in types) {
    subBase <- Database[grep(j, Database$HotspotBelonging), ]
    x <- list()
    for (cond in conditions) {
      x[[cond]] <- subBase[grep(cond, subBase$HotspotBelonging), "hotspotID"]
    }

    rpm <- read.table(file.path(results, "rpm_hgtgtsHeatmap.csv"), header = TRUE, sep = ",", row.names = 1)

    venn_postRIDER <- function(sublist1, sublist2, name1, name2, cols) {
      temp1 <- rpm[sapply(setdiff(sublist1, sublist2), toString), cols, drop = FALSE]
      temp2 <- rpm[sapply(intersect(sublist1, sublist2), toString), cols, drop = FALSE]
      temp3 <- rpm[sapply(setdiff(sublist2, sublist1), toString), cols, drop = FALSE]
      filtered <- read.table(file.path(plots, "filtered", paste0("filtered_Ide_", name1, name2, ".csv")),
                             header = TRUE, row.names = 1, sep = ",")
      j2 <- list(
        union(rownames(temp1)[rownames(temp1) %in% rownames(filtered)],
              rownames(temp2)[rownames(temp2) %in% rownames(filtered)]),
        union(rownames(temp2)[rownames(temp2) %in% rownames(filtered)],
              rownames(temp3)[rownames(temp3) %in% rownames(filtered)])
      )
      names(j2) <- c(name1, name2)
venn_counts <- c(
  setdiff1 = length(setdiff(j2[[1]], j2[[2]])),
  setdiff2 = length(setdiff(j2[[2]], j2[[1]])),
  intersect12 = length(intersect(j2[[1]], j2[[2]]))
)
names(venn_counts) <- c(name1, name2, paste0(name1, "&", name2))
venn <- euler(venn_counts)

      cairo_pdf(file.path(plots, paste0("venn_", name1, "vs", name2, "_filtered.pdf")), width = 6, height = 6)
      plot(venn, quantities = TRUE, main = paste("Idelalisib Hotspot -", name1, "vs", name2, "- Threshold", jamesonFirst))
      dev.off()

      write.table(unique(union(j2[[1]], j2[[2]])), file.path(res2, paste0(name2, "Filtered.csv")), col.names = FALSE, row.names = FALSE)
      write.table(intersect(j2[[1]], j2[[2]]), file.path(res2, paste0(toupper(name2), "commonFiltered.csv")), col.names = FALSE, row.names = FALSE)

      return(j2)
    }

    j2Filtered <- list()
    j2Filtered$DMSO <- venn_postRIDER(x$DMSO, x$Taze, "DMSO", "Taze", c(1,2))$DMSO
    j2Filtered$Taze <- venn_postRIDER(x$DMSO, x$Taze, "DMSO", "Taze", c(1,2))$Taze
    j2Filtered$Vale <- venn_postRIDER(x$DMSO, x$Vale, "DMSO", "Vale", c(1,3))$Vale

    venn_all <- euler(c(
      "DMSO" = length(setdiff(setdiff(j2Filtered$DMSO, j2Filtered$Taze), j2Filtered$Vale)),
      "Taze" = length(setdiff(setdiff(j2Filtered$Taze, j2Filtered$DMSO), j2Filtered$Vale)),
      "Vale" = length(setdiff(setdiff(j2Filtered$Vale, j2Filtered$Taze), j2Filtered$DMSO)),
      "DMSO&Taze&Vale" = length(Reduce(intersect, j2Filtered)),
      "DMSO&Taze" = length(setdiff(intersect(j2Filtered$DMSO, j2Filtered$Taze), j2Filtered$Vale)),
      "DMSO&Vale" = length(setdiff(intersect(j2Filtered$DMSO, j2Filtered$Vale), j2Filtered$Taze)),
      "Taze&Vale" = length(setdiff(intersect(j2Filtered$Taze, j2Filtered$Vale), j2Filtered$DMSO))
    ))
    cairo_pdf(file.path(plots, "venn_all_filtered.pdf"), width = 6, height = 6)
    plot(venn_all, quantities = TRUE, main = paste("Idelalisib Hotspot - All - Threshold", jamesonFirst))
    dev.off()

    bar <- c(length(j2Filtered$DMSO), length(j2Filtered$Taze), length(j2Filtered$Vale))
    names(bar) <- c("DMSO", "Taze", "Vale")
    write.table(t(bar), file.path(plots, "filtered", "barplot.csv"), col.names = NA)
  }
}



--- RIDER Filtering & Venn Analysis - Threshold 5 ---


ERROR: Error in a[sapply(setdiff(j2$DMSO, j2$Tazemetostat), toString), c(1, 2), : invalid subscript type 'list'


In [ ]:
results="/sharedFolder/Results/Hotspot/Revision1/Idelalisib/FULL"
previousResults="/sharedFolder/Results/Hotspot/Revision1/FULL/5/"
path="/sharedFolder/Data/1_HTGTS/13_Revision1/"
res2=paste(results,"HotspotIDList/",sep="")


hotspotpath=previousResults

plots=paste(results,"/rpmPlot/",sep="")
dir.create(plots)
# centering with 'scale()'
center_scale <- function(x) {
    scale(x, scale = FALSE)
}
library("pheatmap")
library("vioplot")
library("ggVennDiagram");
library("ggplot2")
library("GenomicRanges")
rider2="/sharedFolder/rider/RIDER_v0.2.jar"
Database=read.table(paste(previousResults,"Database.csv",sep="/"),header=TRUE,sep=",")
gg=GRanges(seqnames=as.matrix(Database[,1]),ranges=IRanges(start = as.numeric(Database[,2]),end=as.numeric(Database[,3])))    
library(eulerr)
filePlots=list.files(plots,pattern=".csv")
len <- 3
toRemoveList <- vector(mode = "list", length = len)
names(toRemoveList)=filePlots

a=list.files(path)[grep("Idelalisib",list.files(path))]
b=list.dirs(hotspotpath,recursive=FALSE)
b=b[grep("Idelalisib",b)]

conditions=c("DMSO","Taze","Vale")
dir.create(paste(plots,"filtered",sep="/"))

for(i in conditions){
    oneFile=a[grep(i,a)]
    oneFileS=b[grep(i,b)]
    pathone=list.dirs(oneFileS,recursive=FALSE)
    oneFileS=list.files(list.dirs(oneFileS,recursive=FALSE),pattern="bed")
    oneFileS=paste(pathone,oneFileS,sep="/")
    for(j in conditions){
        if(!((i=="Taze" && j =="Vale") || (j=="Taze" && i =="Vale"))){
        twoFile=a[grep(j,a)]
        twoFileS=b[grep(j,b)]
        pathtwo=list.dirs(twoFileS,recursive=FALSE)
        twoFileS=list.files(list.dirs(twoFileS,recursive=FALSE),pattern="bed")
        twoFileS=paste(pathtwo,twoFileS,sep="/")
        if(i!=j){
            count=intersect(grep(i,names(toRemoveList)),grep(j,names(toRemoveList)))
            toRemove=c()
            rpmPlot=read.table(paste(plots,"/",intersect(filePlots[grep(i,filePlots)],filePlots[grep(j,filePlots)]),sep=""),header=TRUE,sep=",",row.names=1)
            system(paste("java -jar ",rider2," -task ValSite -s_file ",oneFileS," ",twoFileS," -e_file ",path,"/",oneFile," -c_file ",path,"/",twoFile," -genome hg38 -out_dir ",plots,"/",i,"_",j,sep=""))
            system(paste("chmod -R 777 ",plots,"/",i,"_",j,sep=""))
            compare=read.table(paste(plots,"/",i,"_",j,"/",list.files(paste(plots,"/",i,"_",j,sep=""),pattern=".txt"),sep=""),header=TRUE,sep="\t")
            for(n in which(compare[,16]>0.05)){
                tempGG=GRanges(seqnames=as.matrix(compare[n,2]),ranges=IRanges(start = as.numeric(compare[n,3]),end=as.numeric(compare[n,4])))    
                toRemove=append(toRemove,unique(Database[findOverlaps(tempGG,gg)@to,7]))
                }
            toRemove=intersect(sapply(toRemove,FUN=toString),rownames(rpmPlot))
            print(count)
            print(toRemove)
            if(length(toRemoveList[[count]])==0){
            toRemoveList[[count]]=append(toRemoveList[[count]],toRemove)
            }else{
            toRemoveList[[count]]=intersect(toRemoveList[[count]],toRemove)
        }
            }
    }
    }

}

for(i in filePlots){
rpmPlot=read.table(paste(plots,"/",i,sep=""),header=TRUE,sep=",",row.names=1)
rpmPlot=rpmPlot[!(row.names(rpmPlot) %in% toRemoveList[[i]]),]
write.table(rpmPlot,paste(plots,"/filtered/filtered_",i,sep=""),col.names=NA,sep=",")
}
#Venn diagram plot. Check how many hotspot are in common
Database=read.table(paste(previousResults,"Database.csv",sep="/"),header=TRUE,sep=",")
types=c("Idelalisib")
conditions=c("DMSO","Taze","Vale")
for(j in types){
subBase=Database[grep(j,Database[,6]),]
x <- list()
count=1
for(i in conditions){
x[[count]]=subBase[grep(i,subBase[,6]),7]
count=count+1
}
names(x)=conditions
system("echo yeah")
}
jjj=(unlist(x))
j2=x
mj2=list()
count=1
a=as.matrix(read.table(paste(results,"rpm_hgtgtsHeatmap.csv",sep="/"),header=TRUE,row.names=1,sep=","))
a2=as.matrix(read.table(paste(results,"/rpmPlot/filtered/filtered_Ide_DMSOTaze.csv",sep=""),sep=",",header=TRUE,row.names=1))
temp=a[sapply(setdiff(j2$DMSO,j2$Taze),toString),c(1,2)]
if(length(temp)>2){
temp=temp[sort(temp[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp2=a[sapply(intersect(j2$DMSO,j2$Taze),toString),c(1,2)]
if(length(temp2)>2){
temp2=temp2[sort(temp2[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp3=a[sapply(setdiff(j2$Taze,j2$DMSO),toString),c(1,2)]
if(length(temp3)>2){
temp3=temp3[sort(temp3[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}

mj2[[count]]=temp[intersect(rownames(a2),rownames(temp)),]
count=count+1
mj2[[count]]=temp2[intersect(rownames(a2),rownames(temp2)),]

count=count+1
mj2[[count]]=temp3[intersect(rownames(a2),rownames(temp3)),]

xTaze=list()
xTaze[[1]]=union(rownames(mj2[[1]]),rownames(mj2[[2]]))
xTaze[[2]]=union(rownames(mj2[[2]]),rownames(mj2[[3]]))
names(xTaze)=c("DMSO","Taze")

j3=xTaze

fit1 <- euler(c("DMSO" = length(setdiff(j3$DMSO,j3$Taze)), "Taze" = length(setdiff(j3$Taze,j3$DMSO)),"DMSO&Taze"=length(intersect(j3$DMSO,j3$Taze)) ))
plot(fit1,quantities = TRUE,main="Idelalisib Hotspot")
count=count+1



temp=a[sapply(setdiff(j2$DMSO,j2$Vale),toString),c(1,3)]
if(length(temp)>2){
temp=temp[sort(temp[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp2=a[sapply(intersect(j2$DMSO,j2$Vale),toString),c(1,3)]
if(length(temp2)>2){
temp2=temp2[sort(temp2[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp3=a[sapply(setdiff(j2$Vale,j2$DMSO),toString),c(1,3)]
if(length(temp3)>2){
temp3=temp3[sort(temp3[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}

a2=as.matrix(read.table(paste(results,"/rpmPlot/filtered/filtered_Ide_DMSOVale.csv",sep=""),sep=",",header=TRUE,row.names=1))

mj2[[count]]=temp[intersect(rownames(a2),rownames(temp)),]
count=count+1
mj2[[count]]=temp2[intersect(rownames(a2),rownames(temp2)),]

count=count+1
mj2[[count]]=temp3[intersect(rownames(a2),rownames(temp3)),]
xVale=list()
xVale[[1]]=union(rownames(mj2[[1+3]]),rownames(mj2[[2+3]]))
xVale[[2]]=union(rownames(mj2[[2+3]]),rownames(mj2[[3+3]]))
names(xVale)=c("DMSO","Vale")
j3=xVale

fit1 <- euler(c("DMSO" = length(setdiff(j3$DMSO,j3$Vale)), "Vale" = length(setdiff(j3$Vale,j3$DMSO)),"DMSO&Vale"=length(intersect(j3$DMSO,j3$Vale)) ))
plot(fit1,quantities = TRUE,main="Idelalisib Hotspot")
 write.table(unique(union(xTaze$DMSO,xTaze$Taze)),paste(res2,"/TazeFiltered.csv",sep=""),col.names=FALSE,row.names=FALSE)
write.table(unique(union(xVale$Vale,xVale$DMSO)),paste(res2,"/ValeFiltered.csv",sep=""),col.names=FALSE,row.names=FALSE)
write.table(intersect(xTaze$DMSO,xTaze$Taze),paste(res2,"/TAZEcommonFiltered.csv",sep=""),col.names=FALSE,row.names=FALSE)
write.table(intersect(xVale$DMSO,xVale$Vale),paste(res2,"/VALEcommonFiltered.csv",sep=""),col.names=FALSE,row.names=FALSE)
j2Filtered=list(union(xTaze$DMSO,xVale$DMSO),xTaze$Taze,xVale$Vale)
 names(j2Filtered)=c("DMSO","Taze","Vale")
fit1 <- euler(c("DMSO" = length(setdiff(setdiff(j2Filtered$DMSO,j2Filtered$Taze),j2Filtered$Vale)), "Taze" = length(setdiff(setdiff(j2Filtered$Taze,j2Filtered$DMSO),j2Filtered$Vale)), "Vale" = length(setdiff(setdiff(j2Filtered$Vale,j2Filtered$Taze),j2Filtered$DMSO)),"DMSO&Taze&Vale"=length(intersect(intersect(j2Filtered$DMSO,j2Filtered$Taze),j2Filtered$Vale)),"DMSO&Taze"=length(setdiff(intersect(j2Filtered$DMSO,j2Filtered$Taze),j2Filtered$Vale)),"DMSO&Vale"=length(setdiff(intersect(j2Filtered$DMSO,j2Filtered$Vale),j2Filtered$Taze)),"Taze&Vale"=length(setdiff(intersect(j2Filtered$Taze,j2Filtered$Vale),j2Filtered$DMSO)) ))
plot(fit1,quantities = TRUE,main="Idelalisib Hotspot")
fit1 <- euler(c("Taze" = length(setdiff(j2Filtered$Taze,j2Filtered$Vale)), "Vale" = length(setdiff(j2Filtered$Vale,j2Filtered$Taze)),"Taze&Vale"=length(intersect(j2Filtered$Vale,j2Filtered$Taze)) ))
plot(fit1,quantities = TRUE,main="Idelalisib Hotspot")
tj=c(length(j2Filtered$DMSO),length(j2Filtered$Taze),length(j2Filtered$Vale))
names(tj)=c("DMSO","Taze","Vale")
write.table(tj,paste(results,"/rpmPlot/filtered/barplot.csv",sep=""),col.names=NA)
names(mj2)=c("DMSO_spec_vsTaze","DMSO_TAZE","TazeSPec","DMSO_spec_vsVale","DMSOVale","ValeSpec")
dir.create(paste(results,"/rpmPlot/filtered/sortNames/",sep=""))
for(i in seq(length(mj2))){
write.table(rownames(mj2[[i]]),paste(results,"/rpmPlot/filtered/sortNames/_",i,"_",names(mj2)[i],".csv",sep=""),col.names=FALSE,row.names=FALSE)
}


In [56]:
annotate=function(data1,data2){
    data2=read.table(data2,header=TRUE,sep=",")

newNames=c()
for(i in rownames(data1)){
    if(unique(data2[which(data2$hotspotID==i),"annotation"])=="NULL"){newNames=append(newNames,paste(unique(data2[which(data2$hotspotID==i),c("Chromosome","ChrStart","ChrEnd")]),collapse="_"))}else if(length(strsplit(unique(data2[which(data2$hotspotID==i),"annotation"]),"_")[[1]])>3){ 
        newNames=append(newNames,paste(unique(data2[which(data2$hotspotID==i),c("Chromosome","ChrStart","ChrEnd")]),collapse="_"))
    }
    else{
newNames=append(newNames,unique(data2[which(data2$hotspotID==i),"annotation"]))}
}
rownames(data1)=newNames
return(data1)
}

thresholds <- c(5, 6, 7)
for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  

path="/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"

a=list.files(path,pattern=".bed",recursive=TRUE)
a=mixedsort(a)
a2=sapply(a,FUN=function(x){strsplit(x,".bed")[[1]][1]})
a2=sapply(a2,FUN=function(x){strsplit(x,"__")[[1]][2]})
print("Ide ANALYSIS")
dir.create(results,recursive=TRUE)
  # Carica e filtra il database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  Database=Database[grep("Ide",Database[,6]),]


a=list.files(path,pattern=".bed",recursive=TRUE)
a=mixedsort(a)
a2=sapply(a,FUN=function(x){strsplit(x,".bed")[[1]][1]})
a2=sapply(a2,FUN=function(x){strsplit(x,"__")[[1]][2]})
a2=c(a2[grep("Ide",a2)],a2[grep("AID-KO",a2)],a2[grep("WT",a2)])
a=c(a[grep("Ide",a)],a[grep("AID-KO",a)],a[grep("WT",a)])

hgtgts_count=matrix(0,ncol=length(unique(Database[,7])),nrow=length(a2))
colnames(hgtgts_count)=sapply(unique(Database[,7]),FUN=toString)
rownames(hgtgts_count)=a2
hgtgts_rpm=hgtgts_count
hgtgts_fpkm=hgtgts_count
for(j in a){
    system(paste("echo",j))
temp=read.table(paste(path,"/",j,sep=""),sep="\t")
tempGG=GRanges(seqnames=as.matrix(temp[,1]),ranges=IRanges(start = as.numeric(temp[,2]),end=as.numeric(temp[,3])))    
    for(i in seq(nrow(Database))){
        system(paste("echo",i/nrow(Database)))
        gg=GRanges(seqnames=as.matrix(Database[i,1]),ranges=IRanges(start = as.numeric(Database[i,2]),end=as.numeric(Database[i,3])))    
        over=countOverlaps(gg,tempGG)
        name=as.vector(noExt(fNumber(j)))
        hgtgts_count[name,toString(Database[i,7])]= over
        hgtgts_rpm[name,toString(Database[i,7])]= ((over)/nrow(temp))*1000000
   }

}
write.table((t(hgtgts_count)),paste(results,"/",jamesonFirst,"/count_hgtgtsHeatmap_WITHEVERYTHING.csv",sep=""),col.names=NA,sep=",")
write.table((t(hgtgts_rpm)),paste(results,"/",jamesonFirst,"/rpm_hgtgtsHeatmap_PlusWITHEVERYTHING.csv",sep=""),col.names=NA,sep=",")
    

#PART 2 

#Venn diagram plot. Check how many hotspot are in common
conditions=c("DMSO","Taze","Vale")
x <- list()
count=1
for(i in conditions){
subBase <- Database
x[[count]]=subBase[grep(i,subBase[,6]),7]
count=count+1
}
names(x)=conditions
system("echo yeah")

    
jjj=(unlist(x))
j2=x 
fit1 <- euler(c("DMSO" = length(setdiff(setdiff(j2$DMSO,j2$Taze),j2$Vale)), "Taze" = length(setdiff(setdiff(j2$Taze,j2$DMSO),j2$Vale)), "Vale" = length(setdiff(setdiff(j2$Vale,j2$Taze),j2$DMSO)),"DMSO&Taze&Vale"=length(intersect(intersect(j2$DMSO,j2$Taze),j2$Vale)),"DMSO&Taze"=length(setdiff(intersect(j2$DMSO,j2$Taze),j2$Vale)),"DMSO&Vale"=length(setdiff(intersect(j2$DMSO,j2$Vale),j2$Taze)),"Taze&Vale"=length(setdiff(intersect(j2$Taze,j2$Vale),j2$DMSO)) ))
DMSOSpec=setdiff(setdiff(x$DMSO,x$Vale),x$Taze)
TazeSpec=setdiff(setdiff(x$Taze,x$DMSO),x$Vale)
ValeSpec=setdiff(setdiff(x$Vale,x$DMSO),x$Taze)
TazVale=setdiff(intersect(x$Taze,x$Vale),x$DMSO)
TazDMSO=setdiff(intersect(x$Taze,x$DMSO),x$Vale)
DMSOVale=setdiff(intersect(x$ Vale,x$DMSO),x$Taze)
common=intersect(intersect(x$ Vale,x$DMSO),x$Taze)
fit1 <- euler(c("DMSO" = length(setdiff(j2$DMSO,j2$Taze)), "Taze" = length(setdiff(j2$Taze,j2$DMSO)),"DMSO&Taze"=length(intersect(j2$DMSO,j2$Taze)) ))
fit1 <- euler(c("Taze" = length(setdiff(j2$Taze,j2$Vale)), "Vale" = length(setdiff(j2$Vale,j2$Taze)),"Taze&Vale"=length(intersect(j2$Vale,j2$Taze)) ))
fit1 <- euler(c("DMSO" = length(setdiff(j2$DMSO,j2$Vale)), "Vale" = length(setdiff(j2$Vale,j2$DMSO)),"DMSO&Vale"=length(intersect(j2$DMSO,j2$Vale)) ))


    
#PART 3 
mj=list()
count=1
dir.create(paste(results,"/",jamesonFirst,"/rpmPlot/",sep=""))
a=t(hgtgts_rpm)
temp=a[sapply(setdiff(j2$DMSO,j2$Taze),toString),]
if(length(temp)>2){
temp=temp[sort(temp[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp2=a[sapply(intersect(j2$DMSO,j2$Taze),toString),]
if(length(temp2)>2){
temp2=temp2[sort(temp2[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp3=a[sapply(setdiff(j2$Taze,j2$DMSO),toString),]
if(length(temp3)>2){
temp3=temp3[sort(temp3[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}

mj[[count]]=temp
count=count+1
mj[[count]]=temp2

count=count+1
mj[[count]]=temp3



count=count+1
Idetaze=rbind(temp,temp2,temp3)
filteredList=rownames(read.table(paste(results,"/",jamesonFirst,"/rpmPlot/filtered/filtered_Ide_DMSOTaze.csv",sep=""),header=TRUE,row.names=1,sep=","))
write.table(annotate(Idetaze,db_path),paste(results,"/",jamesonFirst,"/rpmPlot/Ide_DMSOTaze_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)
Idetaze=Idetaze[filteredList,]
write.table(annotate(Idetaze,db_path),paste(results,"/",jamesonFirst,"/rpmPlot/filtered/filtered_Ide_DMSOTaze_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)





temp=a[sapply(setdiff(j2$DMSO,j2$Vale),toString),]
if(length(temp)>2){
temp=temp[sort(temp[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp2=a[sapply(intersect(j2$DMSO,j2$Vale),toString),]
if(length(temp2)>2){
temp2=temp2[sort(temp2[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}
temp3=a[sapply(setdiff(j2$Vale,j2$DMSO),toString),]
if(length(temp3)>2){
temp3=temp3[sort(temp3[,1],index.return=TRUE,decreasing=TRUE)$ix,]
}


mj[[count]]=temp
count=count+1
mj[[count]]=temp2

count=count+1
mj[[count]]=temp3

count=count+1
write.table(rbind(temp,temp2,temp3),paste(results,"/",jamesonFirst,"/rpmPlot/Ide_DMSOVale_PlusAIDKO.csv",sep=""),col.names=NA,sep=",",quote=TRUE)
Idevale=rbind(temp,temp2,temp3)
filteredList=rownames(read.table(paste(results,"/",jamesonFirst,"/rpmPlot/filtered/filtered_Ide_DMSOVale.csv",sep=""),header=TRUE,row.names=1,sep=","))
write.table(annotate(Idevale,db_path),paste(results,"/",jamesonFirst,"/rpmPlot/Ide_DMSOVale_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)
Idevale=Idevale[filteredList,]
write.table(annotate(Idevale,db_path),paste(results,"/",jamesonFirst,"/rpmPlot/filtered/filtered_Ide_DMSOVale_PlusEVERYTHING.csv",sep=""),col.names=NA,sep=",",quote=TRUE)

    
    
    
    
    
    
    
    
    
    
}


--- Processing threshold: 5 ---
[1] "Ide ANALYSIS"


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'rownames': cannot open the connection


In [37]:
results

[1] "/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"

In [34]:
#GENERATING RPM PLOTS RAW



,Chromosome,ChrStart,ChrEnd,Signal2Noise,EventCounts,HotspotBelonging,hotspotID,Category,annotation
,<chr>,<int>,<int>,<dbl>,<int>,<chr>,<int>,<chr>,<chr>
69,chr2,88857470,88861556,331.48505,22,MEC-1_Idelalisib_DMSO,125,AID_off_target,NULL
70,chr6,390960,410647,165.74253,11,MEC-1_Idelalisib_DMSO,128,AID_off_target,IRF4
72,chr8,145047853,145069370,286.28255,19,MEC-1_Idelalisib_DMSO,131,AID_off_target,C8orf33
73,chr11,135055498,135075423,180.81003,12,MEC-1_Idelalisib_DMSO,132,AID_off_target,NULL
74,chr12,133212408,133254800,436.95757,29,MEC-1_Idelalisib_DMSO,133,AID_off_target,ZNF268_ANHX
75,chr13,114320599,114346317,210.94503,14,MEC-1_Idelalisib_DMSO,135,AID_off_target,CHAMP1_LINC01054
83,chr17,58327989,58332211,135.60752,9,MEC-1_Idelalisib_DMSO,98,AID_off_target,TSPOAP1_TSPOAP1-AS1_MIR142
87,chr2,88857470,88861556,132.78449,11,MEC-1_Idelalisib_Tazemetostat,125,AID_off_target,NULL
88,chr3,120721841,120722023,96.57054,8,MEC-1_Idelalisib_Tazemetostat,103,AID_off_target,RABL3


[1] "Ide ANALYSIS"
